# Lecture 3 - Julia - Parametric Types and Methods, Mutable Structs

## Contents
1. [Parametric Types](#parametric-types)
2. [Parametric Methods](#parametric-methods)
3. [Structs are Immutable](#structs-immutable)
4. [Further Reading](#further-reading)

In [1]:
# Useful code snippet for printing type trees
using AbstractTrees
AbstractTrees.children(x::Type) = subtypes(x)

## 1. Parametric Types <a class="anchor" id="parametric-types"></a>

- In the last lecture we saw how to define composite types, or structs. For example, to define a point in 2D Cartesian space we could do:

In [2]:
struct MyPoint
    x
    y
end

- Additionally, we could annotate the struct's fields with types, either abstract or concrete. For example:

In [3]:
struct MyPointReal
    x::Real
    y::Real
end

In [4]:
struct MyPointFloat64
    x::Float64
    y::Float64
end

- Of course, this is a tedious coding style. Don't do this!


- Fortunately, Julia provides powerful machinery for easily defining entire families of types.

### 1.1. Parametric Composite Types 

- Julia's type system is *parametric*: types can take parameters, so that type declarations actually introduce an entire family of new types - one for each possible combination of the parameter values.


- Type parameters are introduced immediately after the type name, surrounded by curly braces:

In [5]:
struct Point{T}
   x::T
   y::T
end

- `T` can be *any* Julia type.


- This declaration defines a new parametric type, `Point{T}`, holding two "coordinates" of type `T`.


- For example, `Point{Float64}` is a concrete type equivalent to the type defined by replacing `T` in the definition of `Point` with `Float64`. That is, `Point{Float64}` is functionally equivalent to the `MyPointFloat64` struct that we defined earlier.

In [6]:
isconcretetype(Point{Float64})

true

In [7]:
isconcretetype(Point)

false

- We see therefore that our parametric definition of `Point{T}` actually declares an unlimited number of types, one for each possible value of `T`, each of which is now a usable concrete type. 

In [8]:
Point{Float64}(1.0, 2.0)

Point{Float64}(1.0, 2.0)

In [9]:
Point{String}("1.0", "2.0")

Point{String}("1.0", "2.0")

- When creating an instance of `Point`, the value of the parameter type `T` can be omitted if it is unambiguous:

In [10]:
Point(1.0, 2.0)

Point{Float64}(1.0, 2.0)

In [11]:
Point("1.0", "2.0")

Point{String}("1.0", "2.0")

- `Point` itself is a valid type object (neither abstract nor concrete!) containing all possible instances of `Point{T}` as subtypes:

In [12]:
Point{Float64} <: Point

true

- However, concrete `Point` types with different values of `T` are never subtypes of each other:

In [13]:
Point{Float64} <: Point{AbstractFloat}

false

- Often, we will want to restrict the possible values of the type parameter `T`. For example, to restrict our coordinates to real numbers, we could have done:

In [19]:
struct PointReal{T<:Real}
   x::T
   y::T
end

In [20]:
PointReal{String}

LoadError: TypeError: in PointReal, in T, expected T<:Real, got Type{String}

- We can also have multiple type parameters for a single parametric type. For example:

In [21]:
struct Person{A<:AbstractString, B<:Integer, C<:Real}
    name::A
    age::B
    height::C
    weight::C
end

### 1.2. Parametric Abstract Types

- Parametric abstract type declarations declare a collection of abstract types, in much the same way:

In [22]:
abstract type Pointy{T} end

- Once again, we have a distinct abstract type for every possible value of `T`.


- We could have declared `Point{T}` to be a subtype of `Pointy{T}`. For example:

In [23]:
struct Point2D{T} <: Pointy{T}
   x::T
   y::T
end

In [24]:
struct Point3D{T} <: Pointy{T}
   x::T
   y::T
   z::T
end

- We now have a distinct type tree for each allowed (real) value of `T`. For example:

In [25]:
print_tree(Pointy{Float64})

Pointy{Float64}
├─ Point2D{Float64}
└─ Point3D{Float64}


In [26]:
print_tree(Pointy{Int32})

Pointy{Int32}
├─ Point2D{Int32}
└─ Point3D{Int32}


In [27]:
print_tree(Pointy)

Pointy
├─ Point2D
└─ Point3D


### 1.3. Performance Considerations

- To allow Julia to compile your code into efficient machine code, it is strongly advised to avoid writing structs with abstract field types. For example, don't do this:

In [28]:
struct slow_struct
    a::AbstractFloat
end

- Parametric types offer a much better alternative:

In [29]:
struct fast_struct{T<:AbstractFloat}
    a::T
end

- The reason for the difference is the Julia compiler's use of type inference to optimise your code. The type of an individual instance of `fast_struct` also contains information about the type of `a`, allowing the Julia compiler to make the necessary optimisations:

In [30]:
typeof(fast_struct(1.0))

fast_struct{Float64}

- On the other hand, the type of an instance of `slow_struct` tell the compiler nothing about the type of `a`:

In [31]:
typeof(slow_struct(1.0))

slow_struct

- Let's look at a straightforward example:

In [2]:
struct NumbersAmbiguous
    a
    b
end

In [3]:
struct NumbersTyped{T<:Real}
    a::T
    b::T
end

In [4]:
function multiply(numbers)
    return numbers.a * numbers.b
end

multiply (generic function with 1 method)

In [5]:
numbers_ambiguous = NumbersAmbiguous(1.0, 2.0)
typeof(numbers_ambiguous)

NumbersAmbiguous

In [6]:
numbers_typed = NumbersTyped(1.0, 2.0)
typeof(numbers_typed)

NumbersTyped{Float64}

In [8]:
using BenchmarkTools
@btime multiply(numbers_ambiguous)

  25.221 ns (1 allocation: 16 bytes)


2.0

In [9]:
@btime multiply(numbers_typed)

  15.712 ns (1 allocation: 16 bytes)


2.0

## 2. Parametric Methods <a class="anchor" id="parametric-methods"></a>

- In the last lecture, we learned about methods and multiple dispatch.


- Type parameters can also be used in method definitions. For example:

In [32]:
function f(x::T, y::T) where T
    return true
end

function f(x, y)
    return false
end

f (generic function with 2 methods)

- What do you think this function does?


- As before, the values of `T` can be restricted to subtypes of a given type. For example:

In [33]:
function f_numeric(x::T, y::T) where {T<:Number}
    return true
end

function f_numeric(x, y)
    return false
end

f_numeric (generic function with 2 methods)

- The method type parameter `T` can also be used inside the function body:

In [34]:
function g(x::T) where T
    return T
end

g (generic function with 1 method)

- What do you think this function does?


- As before, multiple parameters are possible:

In [35]:
function concat_number_to_string(x::S, y::T) where {S<:AbstractString, T<:Number}
    return x * string(y)
end

function concat_number_to_string(x::S, y::T) where {S<:Number, T<:AbstractString}
    return string(x) * y
end

concat_number_to_string (generic function with 2 methods)

## 3. Structs are Not Mutable <a class="anchor" id="structs-immutable"></a>

- Composite objects declared with `struct` are **immutable**; they cannot be modified after construction.

In [36]:
struct my_immutable_struct
    a
end
a = my_immutable_struct("original value")
a.a = "new value"

LoadError: setfield!: immutable struct of type my_immutable_struct cannot be changed

- An immutable object might contain mutable objects, such as arrays, as fields. 


- Those contained objects will remain mutable; only the fields of the immutable object itself cannot be changed to point to different objects.

In [37]:
struct array_struct
    a::Array
end
a = array_struct([1.0, 2.0])

a.a[1] = 3.0  # Modify the contained object
a.a

2-element Vector{Float64}:
 3.0
 2.0

In [38]:
a.a = [3.0, 4.0]  # Try to modify the field itself

LoadError: setfield!: immutable struct of type array_struct cannot be changed

- As is often the case with Julia, the main reason for structs to be immutable is to allow the compiler to allocate memory efficiently.


- However, if you really want a mutable struct, you can simply declare one with the `mutable` keyword:

In [39]:
mutable struct my_mutable_struct
    a
end

In [40]:
a = my_mutable_struct("original value")
a.a = "new value"
a.a

"new value"

## 4. Further Reading <a class="anchor" id="further-reading"></a>
1. [Parametric Types](https://docs.julialang.org/en/v1/manual/types/#Parametric-Types)
2. [Parametric Methods](https://docs.julialang.org/en/v1/manual/methods/#Parametric-Methods)
3. [Mutable Composite Types](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types)
4. [Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/)